In [40]:
# Erick Salas, Ricardo Whaibe, Max Escobar, Mariana Pena, Uriel Castenda, Aramis Quiroz
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [41]:
url = 'https://raw.githubusercontent.com/ermiry/modelos-p2/main/TransporteBalanceado.csv'
data_frame = pd.read_csv(url)

In [42]:
#Funcion que verifica si esta balanceado
def checar_balanceo (oferta, demanda):
  oferta_sum = 0
  demanda_sum = 0
  for i in range(np.size(oferta)):
    oferta_sum += oferta[i]

  for i in range(np.size(demanda)):
    demanda_sum += demanda[i]

  return demanda_sum - oferta_sum

In [43]:
loaded_matrix = data_frame.to_numpy()
matriz = loaded_matrix[:-1,1:-1]
oferta = loaded_matrix[0:-1,-1]
demanda = loaded_matrix[-1,1:-1]

In [44]:
objetivo = matriz.flatten()
restricciones = np.zeros((len(oferta)+len(demanda),len(objetivo)))
print(objetivo)

[10 2 20 11 12 7 9 20 4 14 16 18]


In [45]:
limite = np.shape (matriz)

print ("limite: ", limite)

# verificar si la matriz esta balanceada
is_balanceada = checar_balanceo (oferta, demanda)

# checar donde agregar columna o fila
if (is_balanceada > 0): #Cuando se tiene que agregar una fila ficticia
  fila = []
  for i in range(limite[1]):
      fila.append(0)
  
  matriz = np.append(matriz, [fila], axis = 0)
  oferta = np.append(oferta, [is_balanceada], axis=0)

elif (is_balanceada < 0):
  is_balanceada *= -1
  col = [[0]] * limite[0]
  matriz = np.append (matriz,col, axis=1)
  demanda = np.append (demanda, [is_balanceada], axis=0)

limite = np.shape(matriz)
print ("Demanda ", demanda)
print ("oferta ", oferta)
print ("matriz ", matriz)

limite:  (3, 4)
Demanda  [25 35 35 35]
oferta  [45 55 30]
matriz  [[10 2 20 11]
 [12 7 9 20]
 [4 14 16 18]]


In [46]:
count_objetivo = 0
for i in range(limite[0]):
  for j in range(limite[1]):
    if matriz[i,j] == objetivo[count_objetivo]:
      restricciones[i,count_objetivo] = 1
    count_objetivo += 1

copia = list(objetivo)
for i in range(limite[1]):
  for j in range(limite[0]):
    indice = copia.index(matriz[j,i])
    copia[indice] = -1
    restricciones[limite[0]+i,indice] = 1

print(restricciones)

[[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]]


In [47]:
limiteRestricciones = np.concatenate((oferta,demanda),axis = 0)
print(limiteRestricciones)

[45 55 30 25 35 35 35]


In [48]:
res = linprog(objetivo,None,None,restricciones,limiteRestricciones,None,method='simplex')
print(f'Funcion Objetivo : {res.fun}')

unidades = []
for i,x_val in enumerate(res.x):
  print(f'X{i} = {x_val}')
  unidades.append(x_val)

Funcion Objetivo : 1005.0
X0 = 0.0
X1 = 15.0
X2 = 0.0
X3 = 30.0
X4 = 0.0
X5 = 20.0
X6 = 35.0
X7 = 0.0
X8 = 25.0
X9 = 0.0
X10 = 0.0
X11 = 5.0


/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


In [49]:
res = np.zeros((limite[0],limite[1]))
count = 0
for i in range (limite[0]):
  for j in range (limite[1]):
    res[i, j] = unidades[count]
    count += 1

print (res)

[[ 0. 15.  0. 30.]
 [ 0. 20. 35.  0.]
 [25.  0.  0.  5.]]
